In [1]:
# !pip install mediapipe

In [27]:
import cv2
import mediapipe as mp
import pandas as pd
import time
import os

# Khởi tạo thư viện MediaPipe
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_draw = mp.solutions.drawing_utils

# Mở webcam
cap = cv2.VideoCapture(0)

# Danh sách hành động cần thu thập
actions = ["ngoi_lam_viec", "ngoi_nga_lung", "nam_ngu", "gac_chan", "dung_day", "di_lai"]
data = {action: [] for action in actions}
current_label = None  # Mặc định không ghi dữ liệu
recording = False  # Cờ kiểm soát trạng thái ghi

# Số khung hình cần thu thập mỗi hành động
no_of_frames = 600
frame_count = {action: 0 for action in actions}  # Đếm số frame đã thu thập
fps_limit = 60  # Giới hạn FPS
prev_time = 0

def extract_landmarks(results):
    """Trích xuất thông số khung xương từ MediaPipe."""
    if results.pose_landmarks:
        return [coord for lm in results.pose_landmarks.landmark for coord in (lm.x, lm.y, lm.z, lm.visibility)]
    return []

print("Nhấn số để bắt đầu ghi hành động:")
print("1: Ngồi làm việc | 2: Ngồi ngả lưng | 3: Nằm ngủ | 4: Gác chân | 5: Đứng dậy | 6: Đi lại | SPACE: Dừng ghi | Q: Thoát ")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Giới hạn tốc độ FPS
    curr_time = time.time()
    if curr_time - prev_time < 1 / fps_limit:
        continue
    prev_time = curr_time

    # Xử lý ảnh với MediaPipe
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)

    # Nếu đang ghi, trích xuất dữ liệu
    if recording and current_label is not None:
        lm = extract_landmarks(results)
        if lm:
            data[current_label].append(lm)
            frame_count[current_label] += 1  # Tăng số frame đã thu thập
            print(f"Đang ghi '{current_label.upper()}': {frame_count[current_label]}/{no_of_frames} frames")

            # Hiển thị trạng thái ghi dữ liệu trên màn hình
            cv2.putText(frame, f"Recording: {current_label.upper()} ({frame_count[current_label]}/{no_of_frames})",
                        (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

            # Nếu đủ 600 frames thì tự động dừng ghi
            if frame_count[current_label] >= no_of_frames:
                print(f"Đã thu thập đủ {no_of_frames} frames cho '{current_label.upper()}'!")
                recording = False

    # Vẽ khung xương lên ảnh
    if results.pose_landmarks:
        mp_draw.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                               mp_draw.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=4),
                               mp_draw.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=4))

    # Hiển thị thông tin hành động hiện tại
    if current_label:
        cv2.putText(frame, f"Label: {current_label.upper()}", (10, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Hiển thị hình ảnh
    cv2.imshow("Pose Detection", frame)

    # Chuyển đổi hành động khi nhấn phím
    key = cv2.waitKey(1) & 0xFF
    if key == ord('1'):
        current_label = "ngoi_lam_viec"
        recording = True
    elif key == ord('2'):
        current_label = "ngoi_nga_lung"
        recording = True
    elif key == ord('3'):
        current_label = "nam_ngu"
        recording = True
    elif key == ord('4'):
        current_label = "gac_chan"
        recording = True
    elif key == ord('5'):
        current_label = "dung_day"
        recording = True
    elif key == ord('6'):
        current_label = "di_lai"
        recording = True
    elif key == ord(' '):  # Nhấn SPACE để dừng ghi
        recording = False
        print(f"⏸ Dừng ghi hành động '{current_label.upper()}'")
    elif key == ord('q'):
        break

# Lưu dữ liệu vào file CSV 
for action, lm_list in data.items():
    if lm_list:
        filename = f"{action}.csv"
        df = pd.DataFrame(lm_list)

        # Nếu file đã tồn tại, append dữ liệu vào
        if os.path.exists(filename):
            df.to_csv(filename, mode='a', header=False, index=False)
        else:
            df.to_csv(filename, index=False)

print("Dữ liệu đã lưu thành công!")
cap.release()
cv2.destroyAllWindows()


Nhấn số để bắt đầu ghi hành động:
1: Ngồi làm việc | 2: Ngồi ngả lưng | 3: Nằm ngủ | 4: Gác chân | 5: Đứng dậy | 6: Đi lại | SPACE: Dừng ghi | Q: Thoát 
Đang ghi 'DI_LAI': 1/600 frames
Đang ghi 'DI_LAI': 2/600 frames
Đang ghi 'DI_LAI': 3/600 frames
Đang ghi 'DI_LAI': 4/600 frames
Đang ghi 'DI_LAI': 5/600 frames
Đang ghi 'DI_LAI': 6/600 frames
Đang ghi 'DI_LAI': 7/600 frames
Đang ghi 'DI_LAI': 8/600 frames
Đang ghi 'DI_LAI': 9/600 frames
Đang ghi 'DI_LAI': 10/600 frames
Đang ghi 'DI_LAI': 11/600 frames
Đang ghi 'DI_LAI': 12/600 frames
Đang ghi 'DI_LAI': 13/600 frames
Đang ghi 'DI_LAI': 14/600 frames
Đang ghi 'DI_LAI': 15/600 frames
Đang ghi 'DI_LAI': 16/600 frames
Đang ghi 'DI_LAI': 17/600 frames
Đang ghi 'DI_LAI': 18/600 frames
Đang ghi 'DI_LAI': 19/600 frames
Đang ghi 'DI_LAI': 20/600 frames
Đang ghi 'DI_LAI': 21/600 frames
Đang ghi 'DI_LAI': 22/600 frames
Đang ghi 'DI_LAI': 23/600 frames
Đang ghi 'DI_LAI': 24/600 frames
Đang ghi 'DI_LAI': 25/600 frames
Đang ghi 'DI_LAI': 26/600 frame